In [4]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [5]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [6]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [81]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'

# Generate occupation-specific strings
GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

### Initialize input-output paths

In [211]:
# Manual DAG
input_path = f'{occupation_folder}/{occupation}_M_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_M.csv'

# First Last Task DAG
input_path = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_FLT.csv'

# # Conditioned First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CFLT.csv'

# Partitioned DAG
input_path = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_P.csv'

# # Conditioned Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CP.csv'

In [212]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
tasks_stats

# print stats
#tasks_stats.iloc[:,1:].sum()
dag_df

,source,target,comment
0,"Compute cost of travel and accommodations, usi...",Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
1,"Compute cost of travel and accommodations, usi...","Record and maintain information on clients, ve...","The worker maintaining information on clients,..."
2,Converse with customer to determine destinatio...,Provide customer with brochures and publicatio...,The worker providing brochures and travel info...
3,"Book transportation and hotel reservations, us...",Print or request transportation carrier ticket...,The worker responsible for printing or request...
4,Converse with customer to determine destinatio...,Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
5,Provide customer with brochures and publicatio...,Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
6,Converse with customer to determine destinatio...,"Compute cost of travel and accommodations, usi...",The worker computing the cost of travel and ac...
7,"Compute cost of travel and accommodations, usi...",Provide customer with brochures and publicatio...,The worker providing brochures and travel info...
8,Converse with customer to determine destinatio...,"Record and maintain information on clients, ve...",The worker recording and maintaining informati...
9,"Record and maintain information on clients, ve...",Provide customer with brochures and publicatio...,The worker providing brochures and travel info...


In [213]:
# extract list of tasks and create a dictionary for indexing tasks
tasks_list = tasks_stats['task'].unique()
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Collect payment for transportation and accommodations from customer.',
 1: 'Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.',
 2: 'Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tours costs.',
 3: 'Book transportation and hotel reservations, using computer or telephone.',
 4: 'Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.',
 5: 'Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.',
 6: 'Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.',
 7: 'Record and maintain information on clients, vendors, and travel packages.',
 8: '"Target"'}

In [214]:
def path_exists_to_target(adjacency_matrix, target):
    n = adjacency_matrix.shape[0]
    reachable = {}
    
    def dfs(node):
        if node in reachable:  # If already computed, return the result
            return reachable[node]
        
        if node == target:  # Target node is trivially reachable from itself
            reachable[node] = True
            return True
        
        reachable[node] = False  # Assume not reachable initially to avoid cycles
        for neighbor in range(n):
            if adjacency_matrix[node, neighbor] != 0 and dfs(neighbor):
                reachable[node] = True
                break
        
        return reachable[node]
    
    # run dfs(.) function in a for loop to populate reachable dictionary in full
    for node in range(n):
        dfs(node)

        # once reachable is fully populated, determine final result
        if len(reachable) == n:
            if all(reachable.values()):
                return True
            else:
                return False
    

In [215]:
aux_dict = {value: key for key, value in tasks_dict.items()}
target_node = None
for k, v in tasks_dict.items():
    if v == '"Target"':
        target_node = k
        break

print(f'Target node: {target_node}')
result = path_exists_to_target(adjacency_matrix, target_node)
print("A path exists from every node to the Target:", result)

Target node: 8
A path exists from every node to the Target: True


In [216]:
def count_cycles(adjacency_matrix):
    n = adjacency_matrix.shape[0]
    visited = np.zeros(n, dtype=bool)
    rec_stack = np.zeros(n, dtype=bool)
    cycle_count = 0

    def dfs(v):
        nonlocal cycle_count
        visited[v] = True
        rec_stack[v] = True
        
        for neighbor in range(n):
            if adjacency_matrix[v, neighbor] != 0:
                if not visited[neighbor]:
                    if dfs(neighbor):
                        cycle_count += 1
                elif rec_stack[neighbor]:
                    cycle_count += 1
        
        rec_stack[v] = False
        return cycle_count > 0

    for node in range(n):
        if not visited[node]:
            dfs(node)
    
    return cycle_count // 2  # Each cycle is counted twice, so divide by 2

In [217]:
cycle_count = count_cycles(adjacency_matrix)
print("Number of cycles in the graph:", cycle_count)

Number of cycles in the graph: 4


In [218]:
def find_cycles(adjacency_matrix):
    n = adjacency_matrix.shape[0]
    visited = np.zeros(n, dtype=bool)
    rec_stack = np.zeros(n, dtype=bool)
    all_cycles = []

    def dfs(v, path):
        visited[v] = True
        rec_stack[v] = True
        path.append(v)
        
        for neighbor in range(n):
            if adjacency_matrix[v, neighbor] != 0:
                if not visited[neighbor]:
                    dfs(neighbor, path.copy())
                elif rec_stack[neighbor]:
                    cycle_start_index = path.index(neighbor)
                    cycle = path[cycle_start_index:]
                    if cycle not in all_cycles:  # Avoid duplicates
                        all_cycles.append(cycle)
        
        rec_stack[v] = False
        path.pop()

    for node in range(n):
        if not visited[node]:
            dfs(node, [])

    return all_cycles

In [219]:
cycles = find_cycles(adjacency_matrix)
print("Number of cycles in the graph:", len(cycles))
print("Cycles found:", cycles)

Number of cycles in the graph: 1
Cycles found: [[0, 3, 4]]


In [221]:
tasks_dict

{0: 'Collect payment for transportation and accommodations from customer.',
 1: 'Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.',
 2: 'Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tours costs.',
 3: 'Book transportation and hotel reservations, using computer or telephone.',
 4: 'Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.',
 5: 'Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.',
 6: 'Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.',
 7: 'Record and maintain information on clients, vendors, and travel packages.',
 8: '"Target"'}

In [223]:
dag_df.sort_values(by='source')

,source,target,comment
3,"Book transportation and hotel reservations, us...",Print or request transportation carrier ticket...,The worker responsible for printing or request...
24,"Book transportation and hotel reservations, us...","Plan, describe, arrange, and sell itinerary to...","The worker planning, describing, arranging, an..."
26,Collect payment for transportation and accommo...,"""Target""",Job Completion Indicator
10,Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...",The worker booking transportation and hotel re...
11,Collect payment for transportation and accommo...,Print or request transportation carrier ticket...,The worker printing or requesting transportati...
0,"Compute cost of travel and accommodations, usi...",Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
1,"Compute cost of travel and accommodations, usi...","Record and maintain information on clients, ve...","The worker maintaining information on clients,..."
17,"Compute cost of travel and accommodations, usi...","Plan, describe, arrange, and sell itinerary to...","The worker planning, describing, arranging, an..."
7,"Compute cost of travel and accommodations, usi...",Provide customer with brochures and publicatio...,The worker providing brochures and travel info...
13,"Compute cost of travel and accommodations, usi...",Print or request transportation carrier ticket...,The worker printing or requesting transportati...
